In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Interactive Narupa Client"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This notebook assumes an IMD server is running, and lets you visualise it as a live trajectory with NGLView"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {
    "ExecuteTime": {
     "end_time": "2019-06-06T16:04:25.392952Z",
     "start_time": "2019-06-06T16:04:24.453212Z"
    },
    "extensions": {
     "jupyter_dashboards": {
      "version": 1,
      "views": {
       "grid_default": {
        "hidden": true
       },
       "report_default": {
        "hidden": true
       }
      }
     }
    }
   },
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "7fa4873c21074931b32c8cdbcab0ffe5",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "_ColormakerRegistry()"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "from narupa.app.client import NarupaImdClient\n",
    "from narupa.ase.converter import frame_data_to_ase\n",
    "import numpy as np\n",
    "import nglview"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {
    "ExecuteTime": {
     "end_time": "2019-06-06T16:04:26.428800Z",
     "start_time": "2019-06-06T16:04:26.348751Z"
    },
    "extensions": {
     "jupyter_dashboards": {
      "version": 1,
      "views": {
       "grid_default": {
        "hidden": true
       },
       "report_default": {
        "hidden": true
       }
      }
     }
    }
   },
   "outputs": [],
   "source": [
    "class NGLClient(NarupaImdClient):\n",
    "    def __init__(self, dynamic_bonds=False, *args, update_callback=None, **kwargs):\n",
    "        self._view = None\n",
    "        super().__init__(*args, **kwargs)\n",
    "        self.update_callback = update_callback\n",
    "        self.dynamic_bonds = dynamic_bonds\n",
    "    \n",
    "    @property\n",
    "    def view(self):\n",
    "        if self._view is None or self.dynamic_bonds:\n",
    "            atoms = frame_data_to_ase(\n",
    "                self.first_frame,\n",
    "                topology=True,\n",
    "                positions=False,\n",
    "            )\n",
    "            atoms.set_positions(\n",
    "                np.array(self.latest_frame.particle_positions) * 10\n",
    "            )\n",
    "            self._view = nglview.show_ase(atoms)\n",
    "        return self._view\n",
    "\n",
    "    def _on_frame_received(self, frame_index:int, frame):\n",
    "        super()._on_frame_received(frame_index, frame)\n",
    "        self.view.set_coordinates(\n",
    "            {0: np.array(self.latest_frame.particle_positions) * 10}\n",
    "        )\n",
    "        if self.update_callback is not None:\n",
    "            self.update_callback(self.universe)\n",
    "\n",
    "\n",
    "def _get_structure_string(write_method, suffix='.pdb'):\n",
    "    fname = 'tmp.pdb'\n",
    "    write_method(fname)\n",
    "    with open(fname) as fh:\n",
    "        return fh.read()\n",
    "\n",
    "nglview.adaptor._get_structure_string = _get_structure_string"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {
    "ExecuteTime": {
     "end_time": "2019-06-06T16:04:26.572274Z",
     "start_time": "2019-06-06T16:04:26.482553Z"
    },
    "extensions": {
     "jupyter_dashboards": {
      "version": 1,
      "views": {
       "grid_default": {
        "col": 0,
        "height": 19,
        "hidden": false,
        "row": 11,
        "width": 7
       },
       "report_default": {
        "hidden": false
       }
      }
     }
    }
   },
   "outputs": [],
   "source": [
    "client = NGLClient()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {
    "ExecuteTime": {
     "end_time": "2019-06-06T16:04:27.291228Z",
     "start_time": "2019-06-06T16:04:26.945620Z"
    },
    "extensions": {
     "jupyter_dashboards": {
      "version": 1,
      "views": {
       "grid_default": {
        "col": 0,
        "height": 11,
        "hidden": false,
        "row": 0,
        "width": 12
       },
       "report_default": {
        "hidden": false
       }
      }
     }
    }
   },
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "6d5dd192fc85417fb8e4229a111eed82",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "NGLWidget()"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "client.view"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "extensions": {
   "jupyter_dashboards": {
    "activeView": "grid_default",
    "version": 1,
    "views": {
     "grid_default": {
      "cellMargin": 10,
      "defaultCellHeight": 20,
      "maxColumns": 12,
      "name": "grid",
      "type": "grid"
     },
     "report_default": {
      "name": "report",
      "type": "report"
     }
    }
   }
  },
  "hide_input": false,
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.3"
  },
  "pycharm": {
   "stem_cell": {
    "cell_type": "raw",
    "metadata": {
     "collapsed": false
    },
    "source": []
   }
  },
  "toc": {
   "nav_menu": {},
   "number_sections": true,
   "sideBar": true,
   "skip_h1_title": false,
   "toc_cell": false,
   "toc_position": {},
   "toc_section_display": "block",
   "toc_window_display": false
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
